In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, LSTM, SimpleRNN, GRU
from tensorflow.keras.optimizers import AdamW
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.utils import class_weight
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, roc_auc_score
from imblearn.over_sampling import SMOTE
import matplotlib.pyplot as plt

# Load your dataset
df = pd.read_csv(r"C:\\Users\\user\\Desktop\\Project\\EXPERIMENTS\\revised-data\\audio_features_cp.csv")

# Define feature columns and target column
x_cols = ['mfcc_1', 'mfcc_2', 'mfcc_3', 'mfcc_4', 'mfcc_5', 'mfcc_6', 'mfcc_7', 'mfcc_8', 
          'mfcc_9', 'mfcc_10', 'mfcc_11', 'mfcc_12', 'mfcc_13', 'spectral_centroid', 
          'spectral_bandwidth', 'zero_crossing_rate', 'spectrogram_mean', 'spectrogram_median', 'spectrogram_variance']
y_col = 'label'

# Normalize the feature data to range [0,1]
scaler = MinMaxScaler(feature_range=(0, 1))
df[x_cols] = scaler.fit_transform(df[x_cols])

# Split data into features and labels
X = df[x_cols].values
y = df[y_col].values

# Handle class imbalance using SMOTE
smote = SMOTE(random_state=42)
X, y = smote.fit_resample(X, y)

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train_rnn = X_train.reshape((X_train.shape[0], 1, X_train.shape[1]))
X_test_rnn = X_test.reshape((X_test.shape[0], 1, X_test.shape[1]))

# Compute class weights
class_weights = class_weight.compute_class_weight('balanced', classes=np.unique(y), y=y)
class_weights = {i: class_weights[i] for i in range(len(class_weights))}

# Define callbacks for optimization
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5)

# Define model architectures
def build_gru():
    model = Sequential([
        GRU(128, input_shape=(X_train_rnn.shape[1], X_train_rnn.shape[2]), activation='relu', return_sequences=False,
            kernel_regularizer=tf.keras.regularizers.l2(0.01)),
        Dropout(0.3),
        Dense(64, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.01)),
        Dropout(0.3),
        Dense(1, activation='sigmoid')
    ])
    model.compile(optimizer=Adam(learning_rate=0.0005), loss='binary_crossentropy', metrics=['accuracy'])
    return model

def build_nn():
    model = Sequential([
        Dense(128, input_shape=(X_train.shape[1],), activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.01)),
        Dropout(0.3),
        Dense(64, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.01)),
        Dropout(0.3),
        Dense(1, activation='sigmoid')
    ])
    model.compile(optimizer=Adam(learning_rate=0.0005), loss='binary_crossentropy', metrics=['accuracy'])
    return model

def build_rnn():
    model = Sequential([
        SimpleRNN(50, input_shape=(1, X_train.shape[1]), activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.01)),
        Dropout(0.3),
        Dense(1, activation='sigmoid')
    ])
    model.compile(optimizer=Adam(learning_rate=0.0005), loss='binary_crossentropy', metrics=['accuracy'])
    return model

# Initialize models
gru_model = build_gru()
nn_model = build_nn()
rnn_model = build_rnn()

models = {'GRU': gru_model, 'NN': nn_model, 'RNN': rnn_model}
weights = {'GRU': 0.5, 'NN': 0.3, 'RNN': 0.2}  # Tuned ensemble weights

# Train models
for name, model in models.items():
    print(f"Training {name} model...")
    if name in ['GRU', 'RNN']:
        model.fit(X_train_rnn, y_train, validation_split=0.2, epochs=100, batch_size=32, class_weight=class_weights, 
                   verbose=1)
    else:
        model.fit(X_train, y_train, validation_split=0.2, epochs=100, batch_size=32, class_weight=class_weights, 
                   verbose=1)

# Weighted averaging ensemble
def hybrid_predict(models, weights, X_test, X_test_rnn):
    predictions = []
    for name, model in models.items():
        if name in ['GRU', 'RNN']:
            pred = model.predict(X_test_rnn)
        else:
            pred = model.predict(X_test)
        predictions.append(pred * weights[name])
    return np.sum(predictions, axis=0)

# Evaluate hybrid model
y_pred_prob = hybrid_predict(models, weights, X_test, X_test_rnn)
y_pred = (y_pred_prob > 0.5).astype(int)

accuracy = accuracy_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
roc_auc = roc_auc_score(y_test, y_pred)

print("\nHybrid Model Performance:")
print(f"Accuracy: {accuracy * 100:.2f}%")
print(f"F1 Score: {f1:.2f}")
print(f"Precision: {precision:.2f}")
print(f"Recall: {recall:.2f}")
print(f"ROC AUC: {roc_auc:.2f}")

# Plot training and validation accuracy for debugging
for name, model in models.items():
    history = model.history.history
    plt.plot(history['accuracy'], label=f'{name} Training Accuracy')
    plt.plot(history['val_accuracy'], label=f'{name} Validation Accuracy')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.legend()
    plt.title(f'{name} Training vs Validation Accuracy')
    plt.show()


Training GRU model...
Epoch 1/100


C:\Users\user\AppData\Roaming\Python\Python311\site-packages\keras\src\layers\rnn\rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)
C:\Users\user\AppData\Roaming\Python\Python311\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


67/67 ━━━━━━━━━━━━━━━━━━━━ 3s 8ms/step - accuracy: 0.5138 - loss: 1.7498 - val_accuracy: 0.7477 - val_loss: 1.3392
Epoch 2/100
67/67 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.5921 - loss: 1.2485 - val_accuracy: 0.6822 - val_loss: 1.0242
Epoch 3/100
67/67 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.6512 - loss: 0.9761 - val_accuracy: 0.7308 - val_loss: 0.8574
Epoch 4/100
67/67 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.6703 - loss: 0.8333 - val_accuracy: 0.7121 - val_loss: 0.7691
Epoch 5/100
67/67 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7072 - loss: 0.7537 - val_accuracy: 0.7607 - val_loss: 0.7204
Epoch 6/100
67/67 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7428 - loss: 0.7112 - val_accuracy: 0.7701 - val_loss: 0.6897
Epoch 7/100
67/67 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7520 - loss: 0.6820 - val_accuracy: 0.7738 - val_loss: 0.6628
Epoch 8/100
67/67 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7352 - loss: 0.6635 - val_accuracy: 0.7794 - val_loss: 0.6